In [61]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from salishsea_tools import evaltools as et, viz_tools
import os
import scipy.interpolate as sinterp
import cmocean
import netCDF4 as nc
import xarray as xr

In [80]:
path_run = PATH= '/home/jvalenti/scratch/run_SHEM/tuning/pred_flag' 
SalishSeadf = pd.read_csv('/home/jvalenti/MOAD/analysis-jose/notebooks/SHEM/eval/SalishSea_MesozoopModelvsObsMatches_2015to2019_Eval202111.csv', index_col=0,header=0,parse_dates= ['Date','dtUTC'])

In [81]:
# date_original = SalishSeadf['dtUTC']
# date_altered = []

# for t in date_original:
#     date_altered.append(t.replace(year=2021))


# SalishSeadf['dtUTC'] = date_altered


In [82]:
def make_filename(path_run,start,day=0, var='biol_T', res='h'):
    """Construct path prefix for local SHEM results given date object and paths dict
    """
    date  =  (start + timedelta(days=day))
    folder = date.strftime("%d%b%y").lower()
    prefix = os.path.join(path_run, f'{folder}/')
    fname = []
    try:
        for file in os.listdir(prefix):
            if (var in file) and ('_1'+res) in file:
                fname.append(file)
        if len(fname)>1:
            print('more than one file found') 
        try:
            path = os.path.join(prefix, fname[0])
            return path
        except IndexError:
            print('no file available')
    except FileNotFoundError:
        print('no file available')
    

In [83]:
M_path = '/home/jvalenti/MOAD/'
mask = xr.open_dataset(M_path+'grid2/mesh_mask202108_TDV.nc')

In [84]:
SalishSeadf = SalishSeadf[SalishSeadf.dtUTC>datetime(2018,2,26)]
SalishSeadf = SalishSeadf[SalishSeadf.dtUTC<datetime(2018,8,31)]

In [87]:
total_mine = np.zeros(len(SalishSeadf))
micro_mine = np.zeros(len(SalishSeadf))
meso_mine = np.zeros(len(SalishSeadf))

for n in range(len(SalishSeadf)):
    example = SalishSeadf.iloc[n]
    start = example.dtUTC.to_pydatetime()    
    jj = example.j
    ii = example.i
    k_u = example.k_upper
    k_l = example.k_lower
    e3tt = np.array(mask.e3t_0[0,k_u:k_l+1,jj,ii])
    masktt = np.array(mask.tmask[0,k_u:k_l+1,jj,ii])
    res = 'd'
    var = 'biol_T'
    try:
        meso_mine[n] = sum(np.array(xr.open_dataset(make_filename(path_run,start,var=var,res=res)).mesozooplankton[0,k_u:k_l+1,jj,ii])*e3tt*masktt)/sum(e3tt*masktt)
        micro_mine[n] = sum(np.array(xr.open_dataset(make_filename(path_run,start,var=var,res=res)).microzooplankton[0,k_u:k_l+1,jj,ii])*e3tt*masktt)/sum(e3tt*masktt)
        total_mine[n] = float(meso_mine[n]) + float(micro_mine[n])
    except ValueError or FileNotFoundError:
        total_mine[n]=np.nan
        micro_mine[n]=np.nan
        meso_mine[n]=np.nan

no file available


In [88]:
SalishSeadf['SHEM_mesozooplankton']=meso_mine
SalishSeadf['SHEM_microzooplankton']=micro_mine
SalishSeadf['SHEM_total']=total_mine

In [89]:
SalishSeadf['SHEM_microzooplanktonDI']=(SalishSeadf['SHEM_mesozooplankton'])*SalishSeadf['Z_lower']
SalishSeadf['SHEM_mesozooplanktonDI']=(SalishSeadf['SHEM_mesozooplankton'])*SalishSeadf['Z_lower']
SalishSeadf['SHEM_totalDI']=(SalishSeadf['SHEM_total'])*SalishSeadf['Z_lower']

In [90]:
# define log transform function with slight shift to accommodate zero values
def logt(x):
  return np.log10(x+.001)
# define inverse log transform with same shift
def logt_inv(y):
    return 10**y-.001

In [91]:
#Convert SHEMel values to mg C m-3 by muliplying value * C:N of 5.7 * molecular weight of C
SalishSeadf['L10SHEM_mesozooplankton']=logt(SalishSeadf['SHEM_mesozooplankton']*5.7*12)
SalishSeadf['L10SHEM_mesozooplanktonDI']=logt(SalishSeadf['SHEM_mesozooplanktonDI']*5.7*12/1000)
SalishSeadf['logt_invL10SHEM_mesozooplanktonDI']=logt_inv(SalishSeadf['L10SHEM_mesozooplanktonDI'])

#Convert SHEMel values to mg C m-3 by muliplying value * C:N of 5.7 * molecular weight of C
SalishSeadf['L10SHEM_microooplankton']=logt(SalishSeadf['SHEM_microzooplankton']*5.7*12)
SalishSeadf['L10SHEM_microzooplanktonDI']=logt(SalishSeadf['SHEM_microzooplanktonDI']*5.7*12/1000)
SalishSeadf['logt_invL10SHEM_microzooplanktonDI']=logt_inv(SalishSeadf['L10SHEM_microzooplanktonDI'])

#Convert SHEMel values to mg C m-3 by muliplying value * C:N of 5.7 * molecular weight of C
SalishSeadf['L10SHEM_total']=logt(SalishSeadf['SHEM_total']*5.7*12)
SalishSeadf['L10SHEM_totalDI']=logt(SalishSeadf['SHEM_totalDI']*5.7*12/1000)
SalishSeadf['logt_invL10SHEM_totalDI']=logt_inv(SalishSeadf['L10SHEM_totalDI'])

In [92]:
SalishSeadf

,Unnamed: 0,Key,region_name,Lat,Lon,Date,Z_lower,Z_upper,dtUTC,YD,...,SHEM_totalDI,L10SHEM_mesozooplankton,L10SHEM_mesozooplanktonDI,logt_invL10SHEM_mesozooplanktonDI,L10SHEM_microooplankton,L10SHEM_microzooplanktonDI,logt_invL10SHEM_microzooplanktonDI,L10SHEM_total,L10SHEM_totalDI,logt_invL10SHEM_totalDI
1497,79,022618COW3V21412,San Juan,48.67437,-123.0481,2018-02-26 00:00:00,50.0,0,2018-02-26 22:12:00,57,...,32.217026,1.354048,0.053384,1.129794,1.331994,0.053384,1.129794,1.644181,0.343339,2.203645
1498,83,030118CAMV1058,Whidbey Basin,48.05901,-122.3873,2018-03-01 00:00:00,177.0,0,2018-03-01 18:58:00,60,...,139.510205,1.388078,0.636134,4.325469,1.469465,0.636134,4.325469,1.731697,0.979708,9.542498
1499,85,030118SARAV1321,Whidbey Basin,48.25673,-122.5442,2018-03-01 00:00:00,81.0,0,2018-03-01 21:21:00,60,...,82.764129,1.511966,0.420603,2.632922,1.572703,0.420603,2.632922,1.844419,0.752975,5.661066
1500,86,030118SKETV1405,South Sound,47.15243,-122.6586,2018-03-01 00:00:00,114.0,0,2018-03-01 22:05:00,60,...,87.633089,1.394678,0.451719,2.828558,1.443559,0.451719,2.828558,1.720828,0.777797,5.994103
1501,895,IOS2018096000201,Central Strait of Georgia,49.25000,-123.7550,3/3/2018,395.0,0,2018-03-04 00:13:00,63,...,64.047557,0.741398,0.338115,2.177289,0.746606,0.338115,2.177289,1.045001,0.641658,4.380853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701,1339,UBC9999099001201,Tidal Mixed,48.63000,-123.2430,8/27/2018,138.0,0,2018-08-27 17:35:00,239,...,212.405772,1.617464,0.757408,5.719158,1.805074,0.757408,5.719158,2.022347,1.162252,14.528555
1702,830,082718ELIV1037,Bellingham Bay,48.63795,-122.5694,2018-08-27 00:00:00,109.0,0,2018-08-27 17:37:00,239,...,255.320750,1.835614,0.873093,7.465079,1.962529,0.873093,7.465079,2.204718,1.242167,17.463939
1703,1341,UBC9999099001401,Juan de Fuca,48.30000,-124.0000,8/27/2018,104.0,0,2018-08-28 01:45:00,240,...,114.385302,1.515485,0.532632,3.408041,1.627998,0.532632,3.408041,1.876399,0.893482,7.823955
1704,845,083018MUKV1006,Whidbey Basin,47.97166,-122.3222,2018-08-30 00:00:00,192.0,0,2018-08-30 17:06:00,242,...,304.596139,1.630480,0.913824,8.199190,1.818287,0.913824,8.199190,2.035483,1.318801,20.834376


In [93]:
SalishSeadf.to_csv('evaldata_shem_predflag.csv')

In [94]:
#print(monthlymean['Total'].max())
#print(monthlymean['mod_mesozooplankton'].max()*5.7*12)